In [2]:
import os
os.environ['TRANSFORMERS_CACHE'] = "/mnt/swordfish-pool2/harsha"
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [3]:

import torch
device='cuda' if torch.cuda.is_available() else 'cpu' 
from transformers import RobertaForSequenceClassification, RobertaTokenizer,RobertaConfig
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import Trainer, TrainingArguments
from datasets import load_dataset,load_metric

In [34]:
train_dataset = load_dataset('glue', 'sst2', split='train')
val_dataset = load_dataset('glue', 'sst2', split='validation')
test_dataset = load_dataset('glue', 'sst2', split='test')

Found cached dataset glue (/home/hv2237/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Found cached dataset glue (/home/hv2237/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Found cached dataset glue (/home/hv2237/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [35]:
train_dataset = train_dataset.map(lambda examples: {'labels': examples['label']}, batched=True)
val_dataset = val_dataset.map(lambda examples: {'labels': examples['label']}, batched=True)
test_dataset = test_dataset.map(lambda examples: {'labels': examples['label']}, batched=True)
val_dataset = val_dataset.remove_columns(['label'])
test_dataset = test_dataset.remove_columns(['label'])
train_dataset = train_dataset.remove_columns(['label'])

Loading cached processed dataset at /home/hv2237/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-134b5ffdd978af4d.arrow
Loading cached processed dataset at /home/hv2237/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-fd93ca5f7ad2a80f.arrow
Loading cached processed dataset at /home/hv2237/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-3232f58e26d7662d.arrow


In [6]:
model = RobertaForSequenceClassification.from_pretrained('roberta-large')
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classif

In [39]:
MAX_LENGTH = 128
train_dataset = train_dataset.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=MAX_LENGTH), batched=True)
val_dataset = val_dataset.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=MAX_LENGTH), batched=True)
test_dataset = test_dataset.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=MAX_LENGTH), batched=True)


Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [40]:
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format(type='torch', columns=['input_ids',  'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids',  'attention_mask', 'labels'])

In [41]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
training_args = TrainingArguments(
    output_dir='/mnt/swordfish-pool2/harsha/KD',          #output directory
    learning_rate=1e-4,
    num_train_epochs=5,              
    per_device_train_batch_size=32,                #batch size per device during training
    per_device_eval_batch_size=32,                #batch size for evaluation
    logging_dir='/mnt/swordfish-pool2/harsha/logs',            
    logging_steps=100,
    do_train=True,
    do_eval=True,
    no_cuda=False,
    load_best_model_at_end=True,
    # eval_steps=100,
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=val_dataset,            
    compute_metrics=compute_metrics
)

train_out = trainer.train()


In [11]:
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=16) #prepare dataloader


In [21]:
eval_dataloader = DataLoader(val_dataset, batch_size=16)

In [10]:
import textbrewer
from textbrewer import GeneralDistiller
from textbrewer import TrainingConfig, DistillationConfig
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

In [18]:
roberta_mini = RobertaConfig.from_pretrained('philschmid/roberta-large-sst2')
roberta_mini.num_hidden_layers = 3
roberta_mini.output_hidden_states = True

student_model = RobertaForSequenceClassification(roberta_mini) #, num_labels = 2
student_model.to(device=device)


# bert_config = BertConfig.from_json_file('/content/drive/MyDrive/TextBrewer-master/examples/student_config/bert_base_cased_config/bert_config.json')
# bert_config.output_hidden_states = True
# teacher_model = BertForSequenceClassification(bert_config) #, num_labels = 2
# teacher_model.load_state_dict(torch.load('/content/drive/MyDrive/sst2_teacher_model.pt'))
teacher_model = RobertaForSequenceClassification.from_pretrained('philschmid/roberta-large-sst2', output_hidden_states=True)
teacher_model.to(device=device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
         

In [19]:
num_epochs = 20
num_training_steps = len(train_dataloader) * num_epochs
# Optimizer and learning rate scheduler
optimizer = AdamW(student_model.parameters(), lr=1e-4)

scheduler_class = get_linear_schedule_with_warmup
# arguments dict except 'optimizer'
scheduler_args = {'num_warmup_steps':int(0.1*num_training_steps), 'num_training_steps':num_training_steps}


def simple_adaptor(batch, model_outputs):
    return {'logits': model_outputs.logits, 'hidden': model_outputs.hidden_states}

distill_config = DistillationConfig(
    intermediate_matches=[    
     {'layer_T':0, 'layer_S':0, 'feature':'hidden', 'loss': 'hidden_mse','weight' : 1},
     {'layer_T':6, 'layer_S':2, 'feature':'hidden', 'loss': 'hidden_mse','weight' : 1}])
train_config = TrainingConfig()

distiller = GeneralDistiller(
    train_config=train_config, distill_config=distill_config,
    model_T=teacher_model, model_S=student_model, 
    adaptor_T=simple_adaptor, adaptor_S=simple_adaptor)


with distiller:
    distiller.train(optimizer, train_dataloader, num_epochs, scheduler_class=scheduler_class, scheduler_args = scheduler_args, callback=None)

KeyboardInterrupt: 

In [20]:
student_model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
         

In [31]:
from transformers import TrainingArguments, Trainer
import evaluate

training_args = TrainingArguments(
    output_dir='/mnt/swordfish-pool2/harsha/hpml',          #output directory
    # learning_rate=1e-4,
    # num_train_epochs=50,              
    # per_device_train_batch_size=8,                #batch size per device during training
    per_device_eval_batch_size=16,                #batch size for evaluation
    # logging_dir='/content/drive/MyDrive/logs',            
    # logging_steps=100,
    # do_train=True,
    do_eval=True,
    no_cuda=False,
    # load_best_model_at_end=True,
    # save_strategy="epoch",
    # # eval_steps=100,
    # metric_for_best_model = 'eval_loss',
    # evaluation_strategy="epoch"
)
acc = evaluate.load("accuracy")
import numpy as np
def compute_metrics(eval_pred):
    logits, labels,  = eval_pred
    # print(logits)
    logits = logits[0]
    predictions = np.argmax(logits, axis=-1)
    
    return {
        'accuracy': acc.compute(predictions=predictions, references=labels),
        # 'precision': prec.compute(predictions=predictions, references=labels),
        # 'recall': rec.compute(predictions=predictions, references=labels),
        # 'f1': f1.compute(predictions=predictions, references=labels)
    }

In [32]:
trainer = Trainer(
        model=student_model,  # the instantiated 🤗 Transformers model to be trained
        compute_metrics=compute_metrics,
    )
trainer.evaluate(val_dataset)

Trainer is attempting to log a value of "{'accuracy': 0.7798165137614679}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.9046313166618347,
 'eval_accuracy': {'accuracy': 0.7798165137614679},
 'eval_runtime': 1.5999,
 'eval_samples_per_second': 545.024,
 'eval_steps_per_second': 68.128}

In [33]:
trainer = Trainer(
        model=teacher_model,  # the instantiated 🤗 Transformers model to be trained
        compute_metrics=compute_metrics,
    )
trainer.evaluate(val_dataset)

Trainer is attempting to log a value of "{'accuracy': 0.9644495412844036}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.1399652510881424,
 'eval_accuracy': {'accuracy': 0.9644495412844036},
 'eval_runtime': 15.4166,
 'eval_samples_per_second': 56.562,
 'eval_steps_per_second': 7.07}

In [38]:
from transformers import AutoModel, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("azizbarank/distilroberta-base-sst2-distilled")

distil_model = AutoModelForSequenceClassification.from_pretrained("azizbarank/distilroberta-base-sst2-distilled")

In [42]:
trainer = Trainer(
        model=distil_model,  # the instantiated 🤗 Transformers model to be trained
        compute_metrics=compute_metrics,
    )
trainer.evaluate(val_dataset)

{'eval_loss': 0.33526283502578735,
 'eval_accuracy': 0.9094036697247706,
 'eval_f1': 0.909403550579116,
 'eval_precision': 0.9096565586407572,
 'eval_recall': 0.9096888945019785,
 'eval_runtime': 1.0313,
 'eval_samples_per_second': 845.5,
 'eval_steps_per_second': 105.687}

In [45]:
from textpruner import summary
print(summary(distil_model))


LAYER NAME       	        #PARAMS	     RATIO	 MEM(MB)
--model:         	     82,120,452	   100.00%	  313.27
  --roberta:     	     81,528,322	    99.28%	  311.01
    --embeddings:	     39,001,090	    47.49%	  148.78
    --encoder:   	     42,527,232	    51.79%	  162.23
  --classifier:  	        592,130	     0.72%	    2.26
    --dense:     	        590,592	     0.72%	    2.25
    --out_proj:  	          1,538	     0.00%	    0.01



In [46]:
print(summary(teacher_model))


LAYER NAME       	        #PARAMS	     RATIO	 MEM(MB)
--model:         	    355,362,308	   100.00%	 1355.60
  --roberta:     	    354,310,658	    99.70%	 1351.59
    --embeddings:	     52,001,282	    14.63%	  198.37
    --encoder:   	    302,309,376	    85.07%	 1153.22
  --classifier:  	      1,051,650	     0.30%	    4.01
    --dense:     	      1,049,600	     0.30%	    4.00
    --out_proj:  	          2,050	     0.00%	    0.01



In [47]:
print(summary(student_model))


LAYER NAME       	        #PARAMS	     RATIO	 MEM(MB)
--model:         	     90,841,604	   100.00%	  346.54
  --roberta:     	     89,789,954	    98.84%	  342.52
    --embeddings:	     52,001,282	    57.24%	  198.37
    --encoder:   	     37,788,672	    41.60%	  144.15
  --classifier:  	      1,051,650	     1.16%	    4.01
    --dense:     	      1,049,600	     1.16%	    4.00
    --out_proj:  	          2,050	     0.00%	    0.01

